In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import os
import tempfile

directory = "/content/drive/MyDrive/Colab Notebooks/ELEC4840/Project/Ensemble/"

root_dir = tempfile.mkdtemp() if directory is None else directory
os.chdir(root_dir)
os.listdir('.')

['data', '__pycache__', 'models', 'utils', '2016_Ensemble.ipynb']

In [3]:
import torch
from torchsummary import summary

In [5]:
from utils.setup import *
from utils.training import *
from utils.visualise import *

from models.models_resnet import *
from models.models_alexnet import *
from models.models_vgg19 import *

In [6]:
setup_directory("/content/drive/MyDrive/Colab Notebooks/ELEC4840/Project/Ensemble/")

Train folder exists: True
Val folder exists: True
Test folder exists: True


In [7]:
print("Number of files in 'train':", count_files('./data/train'))
print("Number of files in 'val':", count_files('./data/val'))
print("Number of files in 'test':", count_files('./data/test'))

Number of files in 'train': 862
Number of files in 'val': 322
Number of files in 'test': 379


In [7]:
root_train, root_val, root_test = './data/train', './data/val', './data/test'
loader_train, loader_val, loader_test = get_train_test_set(root_train, root_val, root_test)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
model_2017 = AlexNet(input_channel=3, n_classes=2)

model_2017.load_state_dict(torch.load("models/2017_alexnet_model.pth", map_location=torch.device('cpu')))

for param in model_2017.parameters():
    param.requires_grad = False

model_2017.to(device)

summary(model_2017, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

In [15]:
model_2018 = VGG19()

model_2018.load_state_dict(torch.load("models/2018_vgg19_model.pth", map_location=torch.device('cpu')))

for param in model_2018.parameters():
    param.requires_grad = False

model_2018.to(device)

summary(model_2018, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [16]:
resnet50 = get_resnet50(pre_trained=True)
model_2019 = ClassificationModel(resnet50, 2)

model_2019.load_state_dict(torch.load("models/2019_resnet_model.pth", map_location=torch.device('cpu')))

for param in model_2019.parameters():
    param.requires_grad = False

model_2019.to(device)

summary(model_2019, input_size=(3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 133MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
from tqdm import tqdm

y_pred1 = []
y_pred2 = []
y_pred3 = []
y_true = []

with torch.no_grad():
    for inputs, labels in tqdm(loader_train):
        inputs = inputs.to(device)
        labels = labels.to(device)

        pred1 = model_2017(inputs)[:, 0]
        pred2 = model_2018(inputs)[:, 0]
        pred3 = model_2018(inputs)[:, 0]

        y_pred1.extend(np.around(pred1.cpu().numpy()))
        y_pred2.extend(np.around(pred2.cpu().numpy()))
        y_pred3.extend(np.around(pred3.cpu().numpy()))
        y_true.extend(labels.cpu().numpy())

        y_pred1 = [0 if pred < 0 else pred for pred in y_pred1]
        y_pred2 = [0 if pred < 0 else pred for pred in y_pred2]
        y_pred3 = [0 if pred < 0 else pred for pred in y_pred3]

100%|██████████| 54/54 [00:47<00:00,  1.13it/s]


In [30]:
from scipy.stats import mode

maxvoted_pred = []
for i in range(len(y_pred1)):
    ensemble_pred = mode([y_pred1[i], y_pred2[i], y_pred3[i]])[0]
    maxvoted_pred.append(ensemble_pred)

accuracy_maxvoted = accuracy_score(y_true, maxvoted_pred)
auc_maxvoted = roc_auc_score(y_true, maxvoted_pred)

print("Max Voted Ensemble Accuracy:", accuracy_maxvoted)
print("Max Voted Ensemble AUC:", auc_maxvoted)

Max Voted Ensemble Accuracy: 0.8951044083526681
Max Voted Ensemble AUC: 0.9111293591445944
